## Goal: understanding contributors

The term "community" in this context refers to the group of people contributing to Mozilla projects. Thus, this goal could be summarized as characterizing Mozilla community based on their contributors. A contributor will be understood as a person who performs an action that can be tracked in the set of considered data sources. For example: sending a commit, opening or closing a ticket. As they will be different depending on the data source, particular actions used in each analysis will be detailed within particular goals.

The main objective of this goal is to determine a set of characteristics of contributors:

  * Projects: to which projects they contribute.
  * Organizations: to which organizations they are affiliated
  * Gender: which one is their gender
  * Age: which one is their "age" in the project (time contributing)
  * Geographical origin: where do they come from

Those goals can be refined in the following questions:

**Questions**:

* Which projects can be identified?
* Which contributors have activity related to each project?
* Which organizations can be identified?
* Which contributors are affiliated to each organization?
* Which of those contributors are hired by Mozilla, and which are not?
* Which gender are contributors?
* How long have been contributors contributing?
* Where do contributors come from?

These questions can be answered with the following metrics/data:

**Metrics**:

* List of projects
* Contributors by project
* Number of contributors by project over time
* List of organizations
* Contributors by organization
* Number of contributors by organization over time
* Contributors by groups: hired by Mozilla, the rest
* Contributors by gender
* Number of contributors by gender over time
* Time of first and last commit for each contributor
* Length of period of activity for each contributor
* Contributors by time zone (when possible)
* Contributors by city name (when possible)

All the characeterizations of developers (by project, by organization, by hired by Mozilla/rest, by gender, by period of activity, by time zone, by city name) can be a discriminator / grouping factor for the metrics defined for the next goals. Most of these metrics can be made particular for each of the considered data sources.

### Metric Calculations
First we need to load a connection against the proper ES instance. We use an external module to load credentials from a file that will not be shared. If you want to run this, please use your own credentials, just put them in a file named '.settings' (in the same directory as this notebook) following the example file 'settings.sample'.

**TODO**: Add bot and merges filtering.

In [52]:
import pandas

from util import ESConnection
from elasticsearch_dsl import Search

es_conn = ESConnection()

#### List of Projects

To get the list of projects we will query ES to retrieve the unique count of commits for each project. To do that, we bucketize data based on 'project' field (to a maximum of 100 projects, given by 'size' parameter set below).

In [53]:
s = Search(using=es_conn, index='git')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('projects', 'terms', field='project', size=100)\
    .metric('commits', 'cardinality', field='hash', precision_threshold=100000)
result = s.execute()

# In case you need to check response, uncomment line below
#print(result.to_dict()['aggregations'])


In [54]:
df = pandas.DataFrame()

df = df.from_dict(result.to_dict()['aggregations']['projects']['buckets'])
df = df.drop('doc_count', axis=1)
df['commits'] = df['commits'].apply(lambda row: row['value'])
df=df[['key', 'commits']]
df.columns = ['Project', '# Commits']

df

,Project,# Commits
0,mozilla,1902323
1,mozilla-services,147526
2,rust-lang,135992
3,servo,78163
4,mdn,12750
5,moztw,8618
6,mozilla-mobile,9105
7,aframevr,7160
8,mozilla-japan,5787
9,mozmar,5415


#### Contributors by Project

In [55]:
s = Search(using=es_conn, index='git')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('projects', 'terms', field='project', size=100)\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)
result = s.execute()

# In case you need to check response, uncomment line below
#print(result.to_dict()['aggregations'])


In [56]:
df = pandas.DataFrame()

df = df.from_dict(result.to_dict()['aggregations']['projects']['buckets'])
df = df.drop('doc_count', axis=1)
df['contributors'] = df['contributors'].apply(lambda row: row['value'])
df=df[['key', 'contributors']]
df.columns = ['Project', '# Contributors']

df

,Project,# Contributors
0,mozilla,10965
1,mozilla-services,2122
2,rust-lang,2444
3,servo,1384
4,mdn,334
5,moztw,89
6,mozilla-mobile,108
7,aframevr,205
8,mozilla-japan,58
9,mozmar,73


#### Number of contributors by project over time
**TODO**: provide a plot similar to https://analytics.mozilla.community:443/goto/9523b9b00de0b35645de488a1a06514e

In [57]:
s = Search(using=es_conn, index='git')
s.params(timeout=30)

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
s.aggs.bucket('projects', 'terms', field='project', size=10)\
    .bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)

#print(s.to_dict())
result = s.execute()

# In case you need to check response, uncomment line below
#print(result.to_dict()['aggregations'])

In [58]:
df = pandas.DataFrame(columns=['Project', 'Time', '# Contributors'])

result.to_dict()['aggregations']['projects']['buckets']
for b in result.to_dict()['aggregations']['projects']['buckets']:
    #print(b['key'])
    for i in b['time']['buckets']:
        #print("\tdate: ", i['key_as_string'], 'value: ',i['contributors']['value'])
        df.loc[len(df)] = [b['key'], i['key_as_string'], i['contributors']['value']]  

df

,Project,Time,# Contributors
0,mozilla,2015-04-01T00:00:00.000Z,1107.0
1,mozilla,2015-07-01T00:00:00.000Z,1337.0
2,mozilla,2015-10-01T00:00:00.000Z,1500.0
3,mozilla,2016-01-01T00:00:00.000Z,1534.0
4,mozilla,2016-04-01T00:00:00.000Z,1459.0
5,mozilla,2016-07-01T00:00:00.000Z,1433.0
6,mozilla,2016-10-01T00:00:00.000Z,1223.0
7,mozilla,2017-01-01T00:00:00.000Z,1299.0
8,rust-lang,2015-04-01T00:00:00.000Z,298.0
9,rust-lang,2015-07-01T00:00:00.000Z,276.0


#### List of organizations


In [59]:
s = Search(using=es_conn, index='git')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('organizations', 'terms', field='author_org_name', size=100)\
    .metric('commits', 'cardinality', field='hash', precision_threshold=100000)
result = s.execute()

# In case you need to check response, uncomment line below
#print(result.to_dict()['aggregations'])

In [60]:
df = pandas.DataFrame()

df = df.from_dict(result.to_dict()['aggregations']['organizations']['buckets'])
df = df.drop('doc_count', axis=1)
df['commits'] = df['commits'].apply(lambda row: row['value'])
df=df[['key', 'commits']]
df.columns = ['Organization', '# Commits']

df

,Organization,# Commits
0,Mozilla Staff,1371711
1,Community,967091
2,Mozilla Reps,624
3,Unknown,34


#### Contributors by organization

In [61]:
s = Search(using=es_conn, index='git')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('organizations', 'terms', field='author_org_name', size=100).\
    metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)
result = s.execute()

# In case you need to check response, uncomment line below
#print(result.to_dict()['aggregations'])

In [62]:
df = pandas.DataFrame()

df = df.from_dict(result.to_dict()['aggregations']['organizations']['buckets'])
df = df.drop('doc_count', axis=1)
df['contributors'] = df['contributors'].apply(lambda row: row['value'])
df=df[['key', 'contributors']]
df.columns = ['Organization', '# Contributors']

df

,Organization,# Contributors
0,Mozilla Staff,2084
1,Community,13074
2,Mozilla Reps,2
3,Unknown,20


#### Number of contributors by organization over time
**TODO**: provide a plot similar to https://analytics.mozilla.community:443/goto/5dce2b36ec14405a09827860169ae234

#### Contributors by groups: hired by Mozilla, the rest
**TODO**: needs to know who are hired by Mozilla

#### Contributors by gender
**TODO**: Pending of running gender study over the data.

#### Number of contributors by gender over time
**TODO**: Pending of running gender study over the data.

#### Time of first and last commit for each contributor


#### Length of period of activity for each contributor


#### Contributors by time zone (when possible)

#### Contributors by city name (when possible)